<a href="https://colab.research.google.com/github/sumaiya08/medicaldata/blob/master/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

In [0]:
auth.authenticate_user()
#4/zgFrSdkE_AxQq4luovYfjoMlyTKIUwm15-SfE2WS7TwWBu4cBwFK3Zw


In [0]:
project_id='eicudata'


In [0]:
%%bigquery --project $project_id patient
SELECT *
FROM `physionet-data.eicu_crd_demo.patient`


In [0]:
print('Number of patients: \n', len(patient))
patient.head()

Number of patients: 
 1447


,patientunitstayid,patienthealthsystemstayid,gender,age,ethnicity,hospitalid,wardid,apacheadmissiondx,admissionheight,hospitaladmittime24,hospitaladmitoffset,hospitaladmitsource,hospitaldischargeyear,hospitaldischargetime24,hospitaldischargeoffset,hospitaldischargelocation,hospitaldischargestatus,unittype,unitadmittime24,unitadmitsource,unitvisitnumber,unitstaytype,admissionweight,dischargeweight,unitdischargetime24,unitdischargeoffset,unitdischargelocation,unitdischargestatus,uniquepid
0,210014,182373,Male,45,Caucasian,73,89,"Hypertension, uncontrolled (for cerebrovascula...",178.0,13:08:59,-8,,2015,15:00:00,4424,Skilled Nursing Facility,Alive,Neuro ICU,13:16:00,Direct Admit,1,admit,116.0,112.7,15:00:00,4424,Skilled Nursing Facility,Alive,002-10665
1,200026,174624,Male,50,Caucasian,71,87,Ablation or mapping of cardiac conduction pathway,177.8,10:41:00,-311,,2014,18:17:00,1585,Home,Alive,Med-Surg ICU,15:52:00,Operating Room,1,admit,106.1,106.1,17:40:00,1548,Home,Alive,002-10715
2,221131,190993,Male,83,Caucasian,71,87,"Endarterectomy, carotid",175.3,21:43:00,0,,2014,20:00:00,1337,Home,Alive,Med-Surg ICU,21:43:00,Operating Room,1,admit,NaN,72.1,17:46:00,1203,Home,Alive,002-10249
3,221215,191054,Male,49,Caucasian,71,87,"Infarction, acute myocardial (MI)",185.4,03:16:00,-829,,2014,19:46:00,1601,Home,Alive,Med-Surg ICU,17:05:00,Emergency Department,1,admit,145.3,146.6,19:07:00,1562,Home,Alive,002-10627
4,217835,188445,Male,57,Caucasian,73,92,"CABG alone, coronary artery bypass grafting",172.7,01:09:00,-2917,,2014,17:19:00,8133,Home,Alive,CTICU,01:46:00,Operating Room,1,admit,NaN,80.4,08:25:00,4719,Floor,Alive,002-10324
